# Submission

In [16]:
# Magic to automatically update imports if functions in utils are changed
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [875]:
submission_with_whole_train = False

# Feature engineer (option 1)

In [474]:
from sklearn.model_selection import train_test_split

stores_train = pd.read_csv("data/stores_train.csv")
stores_test = pd.read_csv("data/stores_test.csv")
stores_extra = pd.read_csv("data/stores_extra.csv")
if not submission_with_whole_train:
    stores_train, stores_val = train_test_split(stores_train, test_size=0.2, random_state=0)

In [833]:
from utils import split_plaace_cat


store_dataframes = {
    "train": stores_train, 
    "extra": stores_extra, 
    "test": stores_test, 
    }

if not submission_with_whole_train:
    store_dataframes["val"] = stores_val

for df_name, df in store_dataframes.items():
    df = split_plaace_cat(df)

In [477]:
store_dataframes["train"].shape

(10287, 16)

In [478]:
store_dataframes["train"]["log_revenue"] = store_dataframes["train"].revenue.apply(lambda x: np.log1p(x))
if not submission_with_whole_train:
    store_dataframes["val"]["log_revenue"] = store_dataframes["val"].revenue.apply(lambda x: np.log1p(x))

In [1005]:
from utils import create_geographical_columns, create_chain_and_mall_columns, generate_rev_dict, generate_chain_rev_dict, create_mean_chain_rev_col

chain_count = stores_train["chain_name"].value_counts().to_dict()
lower_limit = 1
rev_dict, mean_revenue = generate_rev_dict(store_dataframes["train"])

for df_name, df in tqdm(store_dataframes.items()):
    store_dataframes[df_name] = create_geographical_columns(df)
    store_dataframes[df_name] = create_chain_and_mall_columns(df, chain_count, lower_limit=lower_limit)

chain_rev_dict, log_bounded_chain_rev_dict = generate_chain_rev_dict(store_dataframes["train"])
for df_name, df in tqdm(store_dataframes.items()):
    store_dataframes[df_name] = create_mean_chain_rev_col(df, bounded_chain_revs=chain_rev_dict, log_bounded_chain_revs=log_bounded_chain_rev_dict)


100%|██████████| 4/4 [00:00<00:00, 190.29it/s]


In [480]:
from utils import concat_df_keep_unq_index
concat_df = concat_df_keep_unq_index(stores_train, stores_extra)

!NB next cell takes 3 minutes to run (if comp_plaace_gran. = [1, 2, 3, 4]) (on M1 Mac with 16GB RAM)

(outdated) If you have the .csv files temp_data/closest_comp_\{df_name\}, skip running this cell and run the cell below it instead

In [481]:
from utils import find_dist_to_nearest_comp

nearest_comp_plaace_cat_gran = [1, 2, 3, 4]
n_nearest_comp = [1, 2, 3, 4, 5, 7, 10]

store_dataframes["train"] = find_dist_to_nearest_comp(
    store_dataframes["train"], 
    nearest_comp_plaace_cat_gran, 
    n_nearest_comp, 
    training=True, 
    training_df=concat_df,
    )

In [482]:
if not submission_with_whole_train:
    store_dataframes["val"] = find_dist_to_nearest_comp(
        store_dataframes["val"], 
        nearest_comp_plaace_cat_gran, 
        n_nearest_comp, 
        training=True, 
        training_df=concat_df,
        )

In [483]:
store_dataframes["test"] = find_dist_to_nearest_comp(
    store_dataframes["test"], 
    nearest_comp_plaace_cat_gran, 
    n_nearest_comp, 
    training=False,
    training_df=concat_df
)

In [484]:
comp_plaace_cols = list(store_dataframes["train"].columns[-56:])

In [485]:
dist_dict = store_dataframes["train"][['log_revenue'] + comp_plaace_cols].corr().iloc[0].to_dict()
sorted_relevant_dist_cols = [[k, v] for k, v in sorted(dist_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
sorted_relevant_dist_cols

[['log_revenue', 1.0],
 ['sum_dist_to_nearest_10_comp_plaace_1', -0.11318923733863769],
 ['mean_dist_to_nearest_10_comp_plaace_1', -0.11318923733863753],
 ['mean_dist_to_nearest_7_comp_plaace_1', -0.10947896974172534],
 ['sum_dist_to_nearest_7_comp_plaace_1', -0.10947896974172525],
 ['sum_dist_to_nearest_5_comp_plaace_1', -0.10512637590277266],
 ['mean_dist_to_nearest_5_comp_plaace_1', -0.1051263759027725],
 ['sum_dist_to_nearest_4_comp_plaace_1', -0.10176508975028223],
 ['mean_dist_to_nearest_4_comp_plaace_1', -0.10176508975028223],
 ['sum_dist_to_nearest_3_comp_plaace_1', -0.09554149149809875],
 ['mean_dist_to_nearest_3_comp_plaace_1', -0.09554149149809853],
 ['sum_dist_to_nearest_2_comp_plaace_1', -0.09178689133186166],
 ['mean_dist_to_nearest_2_comp_plaace_1', -0.09178689133186166],
 ['sum_dist_to_nearest_1_comp_plaace_1', -0.07893162376330974],
 ['mean_dist_to_nearest_1_comp_plaace_1', -0.07893162376330974],
 ['sum_dist_to_nearest_2_comp_plaace_2', -0.0522650006821979],
 ['mean_di

In [486]:
comp_relevant_cols = sorted_relevant_dist_cols[1:14:2]
comp_relevant_cols = [r[0] for r in comp_relevant_cols]

In [487]:
comp_relevant_cols

['sum_dist_to_nearest_10_comp_plaace_1',
 'mean_dist_to_nearest_7_comp_plaace_1',
 'sum_dist_to_nearest_5_comp_plaace_1',
 'sum_dist_to_nearest_4_comp_plaace_1',
 'sum_dist_to_nearest_3_comp_plaace_1',
 'sum_dist_to_nearest_2_comp_plaace_1',
 'sum_dist_to_nearest_1_comp_plaace_1']

!NB next cell takes 15 minutes to run. Grab a coffee or somtething while you wait :) (on M1 Mac with 16GB RAM)

If you have the .csv files temp_data/closest_bus_stop_\{df_name\}, skip running this cell and run the cell below it instead

In [488]:
from bus_utils import find_closest_bus_stop

bus_stop_n = [1, 3, 5, 7]
bus_mean = True
bus_sum = True

bus_stop_columns = []

if(bus_sum):
    bus_stop_columns += [f"closest_bus_stop_sum_{i}" for i in bus_stop_n]

if(bus_mean):
    bus_stop_columns += [f"closest_bus_stop_mean_{i}" for i in bus_stop_n]


for df_name, df in tqdm(store_dataframes.items()):
    store_dataframes[df_name] = find_closest_bus_stop(df, bus_stop_n, _sum=bus_sum, _mean=bus_mean)    

100%|██████████| 4/4 [1:12:41<00:00, 1090.36s/it]  


In [489]:
bus_dict = store_dataframes["train"][['log_revenue'] + bus_stop_columns].corr().iloc[0].to_dict()
bus_sorted_relevant_dist_cols = [[k, v] for k, v in sorted(bus_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
bus_sorted_relevant_dist_cols

[['log_revenue', 1.0],
 ['closest_bus_stop_mean_7', -0.04357740220925073],
 ['closest_bus_stop_sum_7', -0.04357740220925051],
 ['closest_bus_stop_mean_5', -0.04101326738679946],
 ['closest_bus_stop_sum_5', -0.04101326738679933],
 ['closest_bus_stop_sum_3', -0.03871598741831423],
 ['closest_bus_stop_mean_3', -0.03871598741831408],
 ['closest_bus_stop_sum_1', -0.031865730956687065],
 ['closest_bus_stop_mean_1', -0.031865730956687065]]

In [490]:
bus_relevant_cols = bus_sorted_relevant_dist_cols[1:9:2]
bus_relevant_cols = [r[0] for r in bus_relevant_cols]

In [491]:
bus_relevant_cols

['closest_bus_stop_mean_7',
 'closest_bus_stop_mean_5',
 'closest_bus_stop_sum_3',
 'closest_bus_stop_sum_1']

In [492]:
from grunnkrets import make_grunnkrets_df

full_population_dataframes = {}
full_pop_columns = []

for df_name, df in tqdm(store_dataframes.items()):
    full_population_dataframes[df_name] = make_grunnkrets_df(df)
    full_pop_columns = full_population_dataframes[df_name].columns

 75%|███████▌  | 3/4 [00:24<00:07,  7.81s/it]/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/grunnkrets.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_population_df[f'{level}.pop_density_log'] = np.log1p(full_population_df[f'{level}.pop_density'])
/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/grunnkrets.py:104: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_population_df[f'{level}.pop_density'] = full_population_df[f'{level}.tot_pop']/full_population_df[f'{level}.area_km2']
/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT

In [493]:
full_pop_columns = list(full_pop_columns[-184:])

In [494]:
for df_name, df in store_dataframes.items():
    store_dataframes[df_name] = df.merge(
        full_population_dataframes[df_name], 
        left_index=True,
        right_index=True,
        how="outer", 
        suffixes=('', '_redundant')
    )
    store_dataframes[df_name].drop(store_dataframes[df_name].filter(regex='_redundant$').columns, axis=1, inplace=True)

In [495]:
full_pop_dict = store_dataframes["train"][['log_revenue'] + full_pop_columns].corr().iloc[0].to_dict()
full_pop_sorted_relevant_dist_cols = [[k, v] for k, v in sorted(full_pop_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
full_pop_sorted_relevant_dist_cols

[['log_revenue', 1.0],
 ['grunnkrets_id.income_density', -0.045215448175719486],
 ['kommune.c_age_19-30_ratio', 0.04472970219247787],
 ['delomrade.mean_age', -0.042862752871162244],
 ['grunnkrets_id.pop_density', -0.0424516238143989],
 ['delomrade.c_age_56-90_ratio', -0.04160424303133776],
 ['delomrade.c_age_19-30_ratio', 0.040181313913752716],
 ['grunnkrets_id.c_age_19-30_ratio', 0.03751481046775187],
 ['kommune.mean_age', -0.031377899056215804],
 ['kommune.c_age_56-90_ratio', -0.029925231776778072],
 ['grunnkrets_id.income_density_log', -0.029019830465243063],
 ['grunnkrets_id.pop_density_log', -0.026966666588406994],
 ['grunnkrets_id.tot_pop_log', 0.026369164197539173],
 ['grunnkrets_id.total_income_log', 0.023533902577560918],
 ['fylke.couple_with_children_income', -0.023220318307684916],
 ['grunnkrets_id.c_age_56-90_ratio', -0.021586672144664712],
 ['kommune.tot_pop_log', 0.01983545484580259],
 ['kommune.total_income_log', 0.019724976569824165],
 ['fylke.single_parent_with_childre

In [496]:
full_pop_relevant_cols = full_pop_sorted_relevant_dist_cols[1:8]
full_pop_relevant_cols = [r[0] for r in full_pop_relevant_cols]

In [497]:
full_pop_relevant_cols

['grunnkrets_id.income_density',
 'kommune.c_age_19-30_ratio',
 'delomrade.mean_age',
 'grunnkrets_id.pop_density',
 'delomrade.c_age_56-90_ratio',
 'delomrade.c_age_19-30_ratio',
 'grunnkrets_id.c_age_19-30_ratio']

In [498]:
fylke_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("fylke.")]
kommune_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("kommune.")]
delomrade_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("delomrade.")]
grunnkrets_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("grunnkrets_id.")]

In [499]:
from utils import mean_func_rev, generate_rev_dict

for i in range(1, 5):
    rev_dict, mean_revenue = generate_rev_dict(stores_train, i)
    for df_name, df in store_dataframes.items():
        df["mean_revenue_" + str(i)] = df["plaace_cat_" + str(i)].apply(lambda x: mean_func_rev(x, rev_dict, mean_revenue))
        df["log_mean_revenue_" + str(i)] = df["mean_revenue_" + str(i)].apply(lambda x: np.log1p(x))

In [500]:
from num_stores import add_num_stores_info

for df_name, df in store_dataframes.items():
    store_dataframes[df_name] = add_num_stores_info(df)

/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/num_stores.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped = geo_df.groupby([ level , cat]).sum()['count'].to_frame()
/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/num_stores.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped = geo_df.groupby([ level , cat]).sum()['count'].to_frame()
/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/num_stores.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns wh

In [501]:
num_store_cols = list(store_dataframes["train"].columns[-64:])

In [502]:
num_store_dict = store_dataframes["train"][['log_revenue'] + num_store_cols].corr().iloc[0].to_dict()
num_store_sorted_relevant_cols = [[k, v] for k, v in sorted(num_store_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
num_store_sorted_relevant_cols

[['log_revenue', 1.0],
 ['fylke.plaace_cat_1_per_capita', 0.3036462055602537],
 ['fylke.plaace_cat_1_per_tot_income', 0.29950542010049447],
 ['kommune.plaace_cat_1_per_capita', 0.25661756586529566],
 ['kommune.plaace_cat_1_per_tot_income', 0.245370813652734],
 ['fylke.plaace_cat_3_per_capita', 0.18362019261601314],
 ['fylke.plaace_cat_1_count', 0.18265093811288075],
 ['fylke.plaace_cat_3_per_tot_income', 0.1797567860196848],
 ['fylke.plaace_cat_4_per_capita', 0.16374528532317106],
 ['fylke.plaace_cat_4_per_tot_income', 0.16261122183977433],
 ['grunnkrets_id.plaace_cat_1_count', 0.1230872564366986],
 ['delomrade.plaace_cat_1_count', 0.11156949047890931],
 ['fylke.plaace_cat_3_count', 0.10050161427908715],
 ['kommune.plaace_cat_3_per_capita', 0.09570990088677313],
 ['kommune.plaace_cat_1_per_km2', 0.08386190923409534],
 ['kommune.plaace_cat_3_per_tot_income', 0.08377451789326158],
 ['fylke.plaace_cat_4_count', 0.07989315137361924],
 ['grunnkrets_id.plaace_cat_1_per_capita', 0.06542117839

In [503]:
num_store_relevant_cols = num_store_sorted_relevant_cols[1:15]
num_store_relevant_cols = [r[0] for r in num_store_relevant_cols]

## Adding mean revenue (plaace_cat, level)

In [894]:
from avg_revenue import add_avg_revenue

for df_name, df in store_dataframes.items():
    store_dataframes[df_name] = add_avg_revenue(df, total=submission_with_whole_train)

In [881]:
mean_rev_cols = list(store_dataframes["train"].columns)[-20:]

In [896]:
mean_rev_dict = store_dataframes["val"][['log_revenue'] + mean_rev_cols].corr().iloc[0].to_dict()
mean_rev_sorted_relevant_cols = [[k, v] for k, v in sorted(mean_rev_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
mean_rev_sorted_relevant_cols

[['log_revenue', 1.0],
 ['country.plaace_cat_4_mean_revenue', 0.5086965809357913],
 ['fylke.plaace_cat_4_mean_revenue', 0.4941024731809247],
 ['country.plaace_cat_3_mean_revenue', 0.4748507632451882],
 ['fylke.plaace_cat_3_mean_revenue', 0.45918831530154497],
 ['kommune.plaace_cat_4_mean_revenue', 0.4366269865474633],
 ['grunnkrets_id.plaace_cat_4_mean_revenue', 0.41545838884788683],
 ['delomrade.plaace_cat_4_mean_revenue', 0.41435511368423755],
 ['kommune.plaace_cat_3_mean_revenue', 0.3894197939511259],
 ['country.plaace_cat_2_mean_revenue', 0.386228220574478],
 ['fylke.plaace_cat_2_mean_revenue', 0.37675915499954626],
 ['delomrade.plaace_cat_3_mean_revenue', 0.34020235836732],
 ['grunnkrets_id.plaace_cat_3_mean_revenue', 0.3268133518022981],
 ['country.plaace_cat_1_mean_revenue', 0.3159747964698619],
 ['fylke.plaace_cat_1_mean_revenue', 0.3093508806098102],
 ['kommune.plaace_cat_2_mean_revenue', 0.29652636200805776],
 ['kommune.plaace_cat_1_mean_revenue', 0.24695499192384032],
 ['del

In [897]:
mean_rev_relevant_cols = mean_rev_sorted_relevant_cols[1:]
mean_rev_relevant_cols = [r[0] for r in mean_rev_relevant_cols]

In [898]:
mean_rev_relevant_log_cols = []
for col in mean_rev_relevant_cols:
    for df_name, df in store_dataframes.items():
        store_dataframes[df_name][col + "_log"] = store_dataframes[df_name][col].apply(lambda x: np.log1p(x))
    mean_rev_relevant_log_cols.append(col + "_log")

# Removing outliers

In [505]:
from outlier_utils import remove_low_revenue

store_dataframes["train"] = remove_low_revenue(store_dataframes["train"], low_rev_limit=0.05)

# Read data directly from CSV (option 2)

## Train, val split

In [890]:
if submission_with_whole_train:
    stores_train = pd.read_csv("temp_data/full_train_features_train.csv", index_col=0)
    stores_extra = pd.read_csv("temp_data/full_train_features_extra.csv", index_col=0)
    stores_test = pd.read_csv("temp_data/full_train_features_test.csv", index_col=0)

    store_dataframes = {
        "train": stores_train, 
        "extra": stores_extra, 
        "test": stores_test, 
        }
else:
    stores_train = pd.read_csv("temp_data/full_features_train.csv", index_col=0)
    stores_val = pd.read_csv("temp_data/full_features_val.csv", index_col=0)
    stores_extra = pd.read_csv("temp_data/full_features_extra.csv", index_col=0)
    stores_test = pd.read_csv("temp_data/full_features_test.csv", index_col=0)

    store_dataframes = {
        "train": stores_train, 
        "extra": stores_extra, 
        "test": stores_test, 
        "val": stores_val
        }

In [533]:
with open("relevant_cols.txt", "r") as f:
    data = f.readlines()

comp_relevant_cols = data[0].strip().split(",")
bus_relevant_cols = data[1].strip().split(",")
num_stores_relevant_cols = data[2].strip().split(",")
full_pop_relevant_cols = data[3].strip().split(",")

# Transforming the data

In [1006]:
from RMSLE import rmsle
from pred_var_utils import reverse_log1p_transform_pred_var
from sklearn.model_selection import GridSearchCV

In [1007]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder


OE_categorical_features = ["bounded_chain_name", "kommune", "delomrade", "is_grocery", "plaace_cat_2", "plaace_cat_3", "plaace_cat_4"]
OE_categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant")),
        ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ]
)

OH_categorical_features = [] #["fylke", "plaace_cat_2"]
OH_categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant")),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)


numerical_features = ["lat", "lon", 
#"mean_revenue_1", "mean_revenue_2", "mean_revenue_3", "mean_revenue_4", 
"log_mean_revenue_1", "log_mean_revenue_2", "log_mean_revenue_3", "log_mean_revenue_4", 
"fylke.plaace_cat_1_mean_revenue_log", #"fylke.plaace_cat_3_mean_revenue_log",
"log_chain_mean_revenue"
] + full_pop_relevant_cols + comp_relevant_cols + bus_relevant_cols + num_stores_relevant_cols #+ mean_rev_relevant_log_cols
numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")), 
        ("scaler", StandardScaler(with_mean=True, with_std=True))]
)


preprocessor = ColumnTransformer(
    transformers=[
        ("oe_cat", OE_categorical_transformer, OE_categorical_features),
        ("oh_cat", OH_categorical_transformer, OH_categorical_features),
        ("num", numerical_transformer, numerical_features),
    ],
    remainder='drop'
)

preprocessor = ColumnTransformer(
    transformers=[
        ("oe_cat", OE_categorical_transformer, OE_categorical_features),
        ("oh_cat", OH_categorical_transformer, OH_categorical_features),
        ("num", numerical_transformer, numerical_features),
    ],
    remainder='drop'
)


X_train = preprocessor.fit_transform(store_dataframes["train"])
if not submission_with_whole_train:
    X_val = preprocessor.transform(store_dataframes["val"])

In [1008]:
y_train = np.array(store_dataframes["train"].log_revenue)
if not submission_with_whole_train:
    y_val = np.array(store_dataframes["val"].revenue)
mean_y = y_train.mean()
std_y = y_train.std()

y_train -= mean_y
y_train /= std_y

# PCA

In [1038]:
PCA_cols_dict = {
    "OE": [],
    "OH": [],  
    "num": []
}

reduntant_cols = ["store_id", "year", "store_name", "plaace_hierarchy_id", "sales_channel_name", "address", "revenue", "log_revenue", "point"]

for col_name, dtype in store_dataframes["train"].dtypes.to_dict().items():
    if(col_name in reduntant_cols):
        continue
    if(dtype == int or dtype == float):
        _type = "num"
    elif(dtype == bool):
        _type = "OE"
    elif(dtype == object):
        if(store_dataframes["train"][col_name].nunique() <= 10):
            _type = "OH"
        else:
            _type = "OE"
    else:
        print(f"Unknown type {dtype} encountered for columns {col_name}")
    PCA_cols_dict[_type].append(col_name)
    

In [619]:
import numpy as np
from sklearn.decomposition import PCA

In [1043]:
preprocessor = ColumnTransformer(
    transformers=[
        ("oe_cat", OE_categorical_transformer, PCA_cols_dict["OE"]),
        ("oh_cat", OH_categorical_transformer, PCA_cols_dict["OH"]),
        ("num", numerical_transformer, PCA_cols_dict["num"]),
    ],
    remainder='drop'
)

In [1044]:
pre_PCA_X_train = preprocessor.fit_transform(store_dataframes["train"])
if not submission_with_whole_train:
    pre_PCA_X_val = preprocessor.transform(store_dataframes["val"])

In [1062]:
pca = PCA(n_components=25)
PCA_X_train = pca.fit_transform(pre_PCA_X_train)
if not submission_with_whole_train:
    PCA_X_val = pca.transform(pre_PCA_X_val)

In [1063]:
print(pca.explained_variance_ratio_)

print(pca.singular_values_)

[5.31746811e-01 3.64934957e-01 6.89630821e-02 1.71733543e-02
 4.45412356e-03 2.34925792e-03 1.38730809e-03 1.12325831e-03
 1.04565042e-03 9.11241691e-04 6.73569806e-04 4.74544966e-04
 3.75263885e-04 3.28965945e-04 3.03755461e-04 2.64302554e-04
 2.32792672e-04 2.27441334e-04 2.12747534e-04 1.67863184e-04
 1.50466531e-04 1.33746089e-04 1.22526464e-04 1.14649294e-04
 1.10844403e-04]
[10463.48700998  8668.25592933  3768.18593069  1880.40569165
   957.64627271   695.48752168   534.4541484    480.91000437
   463.99922111   433.15245776   372.40482392   312.58107322
   277.9664479    260.2552285    250.0840764    233.27847761
   218.93169802   216.40071413   209.29375019   185.90955872
   176.01267646   165.94513996   158.83234293   153.64190538
   151.07092091]


# Training the model

## Random Forest

In [1064]:
from sklearn.ensemble import RandomForestRegressor

# best params (8.11)
# {'max_features': 8, 'min_samples_leaf': 4, 'min_samples_split': 32, 'n_estimators': 500}
# rmsle(on val): 0.719099511243053

rf = RandomForestRegressor(random_state=0, n_jobs=-1, max_features=8, min_samples_leaf=4, min_samples_split=32, n_estimators=500)
rf_params = {
    "n_estimators" : (100, 250, 500), 
    "max_features" : (2, 4, 8), 
    "min_samples_split" : (4, 8, 16, 32), 
    "min_samples_leaf" : (1, 2, 4), 
    }

rf_clf = GridSearchCV(rf, rf_params, verbose=2)

In [1065]:
rf_clf.fit(PCA_X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END max_features=2, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   0.3s
[CV] END max_features=2, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   0.2s
[CV] END max_features=2, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   0.2s
[CV] END max_features=2, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   0.2s
[CV] END max_features=2, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   0.2s
[CV] END max_features=2, min_samples_leaf=1, min_samples_split=4, n_estimators=250; total time=   0.4s
[CV] END max_features=2, min_samples_leaf=1, min_samples_split=4, n_estimators=250; total time=   0.4s
[CV] END max_features=2, min_samples_leaf=1, min_samples_split=4, n_estimators=250; total time=   0.4s
[CV] END max_features=2, min_samples_leaf=1, min_samples_split=4, n_estimators=250; total time=   0.4s
[CV] END m

GridSearchCV(estimator=RandomForestRegressor(max_features=8, min_samples_leaf=4,
                                             min_samples_split=32,
                                             n_estimators=500, n_jobs=-1,
                                             random_state=0),
             param_grid={'max_features': (2, 4, 8),
                         'min_samples_leaf': (1, 2, 4),
                         'min_samples_split': (4, 8, 16, 32),
                         'n_estimators': (100, 250, 500)},
             verbose=2)

In [1067]:
rf_clf.best_params_

{'max_features': 8,
 'min_samples_leaf': 4,
 'min_samples_split': 4,
 'n_estimators': 500}

In [1068]:
rf_clf.best_params_

{'max_features': 8,
 'min_samples_leaf': 4,
 'min_samples_split': 4,
 'n_estimators': 500}

In [1069]:
rf_y_pred = reverse_log1p_transform_pred_var(rf_clf.predict(PCA_X_val), std_y=std_y, mean_y=mean_y)


In [1070]:
rmsle(y_val, rf_y_pred)

0.855060954879145

## Linear Regression

In [1013]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression(n_jobs=-1)

In [1014]:
lr.fit(X_train, y_train)

LinearRegression(n_jobs=-1)

In [1015]:
lr_y_pred = reverse_log1p_transform_pred_var(lr.predict(X_val), std_y, mean_y)
lr_y_pred = np.array([max(0, xi) for xi in lr_y_pred])

In [1016]:
rmsle(y_val, lr_y_pred)

0.736427049127665

## Light GBM

In [1017]:
from lightgbm import LGBMRegressor

# best params (8.11)
# {'learning_rate': 0.05, 'min_child_samples': 16, 'min_split_gain': 0, 'n_estimators': 100, 'num_leaves': 25}
# rmsle(on val): 0.7208999886795342

lgbm = LGBMRegressor(
    random_state=0, 
    n_jobs=-1, 
    learning_rate=0.05, 
    min_child_samples=16, 
    min_split_gain=0, 
    n_estimators=100, 
    num_leaves=25
    )

lgbm_params = {
    "num_leaves" : (10, 25, 31, 75), 
    "learning_rate" : (0.05, 0.1, 0.25), 
    "n_estimators" : (50, 100, 250), 
    "min_split_gain" : (0, 0.01), 
    "min_child_samples" : (4, 8, 16, 32), 
    #"reg_alpha" : (0, 0.01, 0.1), 
    #"reg_lambda" : (0, 0.01, 0.1), 
    }

lgbm_clf = GridSearchCV(lgbm, lgbm_params, verbose=2)

In [1018]:
lgbm.fit(X_train, y_train)

LGBMRegressor(learning_rate=0.05, min_child_samples=16, min_split_gain=0,
              num_leaves=25, random_state=0)

In [751]:
lgbm_clf.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [1019]:
lgbm_y_pred = reverse_log1p_transform_pred_var(lgbm.predict(X_val), std_y, mean_y)
lgbm_y_pred = np.array([max(0, xi) for xi in lgbm_y_pred])

In [1020]:
rmsle(y_val, lgbm_y_pred)

0.7215684121080089

## CatBoost

In [1021]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV


# best params (8.11)
# {'depth': 6, 'l2_leaf_reg': 5, 'learning_rate': 0.05, 'eval_metric': 'RMSE'}
# rmsle(on val) = 0.7190096805219454

cb = CatBoostRegressor(
    random_seed=0, 
    verbose=False, 
    eval_metric="RMSE", 
    depth=6, 
    l2_leaf_reg= 5, 
    learning_rate= 0.05
    )

cb_params = grid = {
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.3],
    'depth': [5, 6, 8, 10],
    'l2_leaf_reg': [4, 5, 6, 7, 8], 
    }

# cb_clf = cb.grid_search(cb_params, X=X_train, y=y_train)

In [961]:
cb_clf["params"]

{'depth': 6, 'l2_leaf_reg': 5, 'learning_rate': 0.05}

In [1022]:
cb.fit(X_train, y_train)

In [1023]:
cb_y_pred = reverse_log1p_transform_pred_var(cb.predict(X_val), std_y, mean_y)
cb_y_pred = np.array([max(0, xi) for xi in cb_y_pred])

In [1024]:
rmsle(y_val, cb_y_pred)

0.719544602165034

## Stacking classifiers

In [1025]:
from sklearn.ensemble import StackingRegressor

estimators = [
    ('rf', rf), 
    ('cb', cb), 
    ('lgbm', lgbm), 
]

rf_end_params = {
    "n_estimators" : (50, 100, 250), 
    "max_features" : (1, 2, 3), 
    "min_samples_split" : (16, 32), 
    "min_samples_leaf" : (2, 4, 8), 
    }

rf_end = RandomForestRegressor(random_state=0, n_jobs=-1, n_estimators=50, max_features=3)

reg = StackingRegressor(
    estimators=estimators,
    final_estimator=rf_end
    )

In [1026]:
reg.fit(X_train, y_train)

StackingRegressor(estimators=[('rf',
                               RandomForestRegressor(max_features=8,
                                                     min_samples_leaf=4,
                                                     min_samples_split=32,
                                                     n_estimators=500,
                                                     n_jobs=-1,
                                                     random_state=0)),
                              ('cb',
                               <catboost.core.CatBoostRegressor object at 0x2dfb4f970>),
                              ('lgbm',
                               LGBMRegressor(learning_rate=0.05,
                                             min_child_samples=16,
                                             min_split_gain=0, num_leaves=25,
                                             random_state=0))],
                  final_estimator=RandomForestRegressor(max_features=3,
                                                        n_estimators=50,
                                                        n_jobs=-1,
                                                        random_state=0))

In [1027]:
reg_y_pred = reverse_log1p_transform_pred_var(reg.predict(X_val), std_y, mean_y)

In [1028]:
rmsle(y_pred=reg_y_pred, y_true=y_val)

0.7529770705254111

## Creating the submission

In [925]:
# Predict on the test set 
X_test = preprocessor.transform(store_dataframes["test"])
y_test_pred = reverse_log1p_transform_pred_var(cb.predict(X_test), std_y, mean_y)

# Generate submission dataframe 
# NOTE: It is important that the ID and predicted values match
submission = pd.DataFrame()
submission['id'] = stores_test.store_id 
submission['predicted'] = np.asarray(y_test_pred)

# Save it to disk (`index=False` means don't save the index in the csv)
submission.to_csv('submission.csv', index=False)

# Create CSV of dataframes

In [1029]:
for df_name, df in store_dataframes.items():
    filepath = Path(f"temp_data/full_features_{df_name}.csv")  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    df.to_csv(filepath, index=True)